In [120]:
import re
import pandas as pd

In [121]:
SEED = 42

In [122]:
prompt_dataset = pd.read_csv('api_dataset.csv')

In [123]:
prompt_dataset.head()

,Unnamed: 0,PROMPT,SCENARIO
0,0,"Giant Biker Gladiator, fullbody; by Peter Mohr...",아이: 저는 거대한 바이커 격투사 그림을 그리고 싶어요. 그리고 그림에는 꼭 Pet...
1,0,crossing the Shadow fold into the pure darknes...,아이: 저번에 본 것처럼 Shadow fold를 그리고 Volcra creature...
2,0,"fire dragon, sitting in dragon nest, dragon, 4...",아이: 용 둥지에 앉은 불타는 용 그림을 그려봐. 그리고 용의 비늘 디테일도 넣어줘...
3,0,"Woody McBride techno rave poster, big yellow s...",아이: 우디 맥브라이드의 테크노 레이브 포스터를 만들어봐. 큰 노란색 스마일리도 추...
4,0,savage barbarian victorian shaman warrior cybo...,아이: 저에게 savage barbarian victorian shaman warr...


In [124]:
def clean_text(dialogue):
    
    dialogue_list = []
    
    for s in dialogue.split('\n'):
        if s != '':
            s = s.strip()
            if s.startswith('\"') and s.endswith('\"'):
                s = s[1:-1]
            dialogue_list.append(s)
    
    return dialogue_list

In [125]:
def contains_english(text):
    # Define the regular expression pattern to match English characters
    pattern = r'[a-zA-Z]'
    
    # Use the search method to check if the pattern is found in the text
    if re.search(pattern, text):
        return True
    else:
        return False

In [126]:
prompt_dataset['SCENARIO'] = prompt_dataset['SCENARIO'].apply(lambda x: clean_text(x))

In [127]:
len(prompt_dataset)

1917

In [128]:
drop_list = []

# 총 1917개
for i, row in prompt_dataset.iterrows():
    if len(row['SCENARIO']) <= 4: # 대화 개수가 4개보다 적은 경우: 324개
        drop_list.append(i)
    
    elif len(row['PROMPT']) < 10: # prompt 길이가 10보다 작은 경우: 21개
        drop_list.append(i)
   
    else:
        for s in row['SCENARIO']: 
            if '아이' in s and 'Model' in s: # 대화가 잘못 섞여들어간 경우: 38개
                drop_list.append(i)
                break
        
            elif '아이' in s and contains_english(s): # 아이의 대화에 영어가 포함된 경우: 613개
                drop_list.append(i)
                break

In [129]:
prompt_dataset.drop(drop_list, inplace=True)

In [130]:
len(prompt_dataset)

921

In [131]:
prompt_dataset.head()

,Unnamed: 0,PROMPT,SCENARIO
6,0,"highly detailed photograph, 4k, of a lobby : :...","[아이: 저는 요즘 브루탈리스트 건축물에 관심이 생겨서, 로비를 만들어봐주세요., ..."
11,0,"child's drawing of the end of the world, crayo...","[아이: 하얀 종이에 세상의 끝을 그려봐. 크레용으로 그리면 좋겠어., Model:..."
14,0,"small texture, black and white, used look, abs...","[아이: 그림을 그려주세요., Model: What kind of picture w..."
15,0,"Victorian era photographs, spirit photography,...",[아이: 요즘 유행하는 영화처럼 귀신들이 나오는 사진을 만들어봐. 그리고 그 사진에...
20,0,Divine syncronicities between Christianity and...,[아이: 성경과 이슬람교의 신비한 유사점을 그림으로 나타내봐. 그리고 그림에는 조상...


In [132]:
from datasets import Dataset

In [133]:
prompt_dataset = Dataset.from_pandas(prompt_dataset)
# Split the dataset into train and test sets (90% train, 10% test)
prompt_dataset = prompt_dataset.train_test_split(test_size=0.1, seed=SEED)

# Split the train set further into train and validation sets (90% train, 10% validation)
prompt_dataset['train'] = prompt_dataset['train'].train_test_split(test_size=0.1)

# Access the resulting datasets
train_dataset = prompt_dataset['train']['train'].to_pandas()
valid_dataset = prompt_dataset['train']['test'].to_pandas()
test_dataset = prompt_dataset['test'].to_pandas()

In [134]:
train_dataset

,Unnamed: 0,PROMPT,SCENARIO,__index_level_0__
0,0,"wide angle 50mm photo of new york flooded, cov...",[아이: 뉴욕을 침수시키고 파괴된 건물 위에 식물을 심어봐. 그리고 건물 안에는 오...,576
1,0,A classical oil painting of the interior of a...,[아이: 초록색 빛이 비치는 고딕식 교회의 내부를 그려봐. 그리고 거기에 새 같은 ...,1305
2,0,"two knights kissing with golden halos, medieva...",[아이: 중세시대 느낌의 그림을 그려봐. 그리고 두 기사가 서로 입맞추고 있는 걸 ...,1884
3,0,"winnie the pooh, soviet poster, highly detaile...","[아이: 유니콘 그림을 그려봐. 그리고 포스터 같은 느낌도 내고 싶어., Model...",939
4,0,"gnome, fantasy, d&d, close-up, portrait by DiT...",[아이: 저는 요즘 판타지 소설을 많이 읽고 있어요. 그래서 판타지 세계를 그린 그...,372
...,...,...,...,...
740,0,"a boy seated playing video games, year 2060, 3...","[아이: 미래의 게임하는 소년이 있는 그림을 그려줘., Model: a boy pl...",992
741,0,concept art Guizhou Plateau Broadleaf and Mixe...,[아이: 제가 보고싶은 그림은 광활한 숲이에요. 그리고 그림이 매우 실감나게 그려지...,1117
742,0,greek goddess selfie in front of temple of oly...,"[아이: 그림 그려줘. 그림은 그리스 신전 앞에서 찍은 사진이어야 해., Model...",1158
743,0,"Comic style, 4D\uff0cwearing suit thug style c...",[아이: 호랑이가 있는 코믹북 같은 그림을 그려봐. 그리고 그 호랑이가 강한 인체를...,1622


In [135]:
def make_accumulate_dataset(df, file_name):
    accumulate_prompts = []
    accumulate_inputs = []
    outputs = []
    accumulate_scenarios = []
    instructions = []
    templates = []

    for i, row in df.iterrows():
        j = 0
        
        while 2 * j + 1 < len(row['SCENARIO']):
            accumulate_prompts.append(row['PROMPT'])
            template_input = '\n'.join(row['SCENARIO'][max(0, 2*j-1): 2*j+1])
            accumulate_inputs.append(template_input)
            outputs.append(row['SCENARIO'][2*j+1])
            accumulate_scenarios.append('\n'.join(row['SCENARIO'][:2*j+2]))
            
            instruction = '아이의 요구사항을 그림 생성을 위한 프롬프트를 번역하세요.' \
                if j == 0 else '기존 프롬프트에 아이의 요구사항을 반영하여 새로운 프롬프트를 생성하세요.'
                 
            instructions.append(instruction)
            
            
            PROMPT_TEMPLATE = [
                "Below is an instruction that describes a task, paired with an input that provides further context.\n",
                "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n\n",
                "Write a response that appropriately completes the request.\n요청을 적절히 완료하는 응답을 작성하세요.\n\n",
                f"### Instruction(명령어):\n{instruction}\n\n### Input(입력):\n{template_input}\n\n### Response(응답):"
                ]
            
            templates.append(''.join(PROMPT_TEMPLATE))
            
            j += 1

    new_dataset = pd.DataFrame()
    new_dataset['template'] = templates
    new_dataset['prompt'] = accumulate_prompts
    new_dataset['instruction'] = instructions
    new_dataset['input'] = accumulate_inputs
    new_dataset['output'] = outputs
    new_dataset['scenario'] = accumulate_scenarios
    
    new_dataset.to_csv(file_name, index=False)
    
    print(f'file name: {file_name}, len: {len(new_dataset)}')

In [136]:
make_accumulate_dataset(train_dataset, 'train.csv')
make_accumulate_dataset(valid_dataset, 'valid.csv')
make_accumulate_dataset(test_dataset, 'test.csv')

file name: train.csv, len: 4016
file name: valid.csv, len: 436
file name: test.csv, len: 530
